# Introduction

Ground measured solar irradiance data is extremely valuable and critical for benchmarking solar radiation products and understanding long-term trends in the radiation budget. However, due to high costs and maintenance requirements, there are only a few hundred high-quality stations globally. Partly due to the scarcity, it has historically been difficult to determine if and where there is a nearby solar radiation monitoring stations. To address this, this site provides an overview of available solar radiation monitoring stations worldwide and supporting metadata.

Note, the site is currently in the draft phase, where only the BSRN stations are included as a demonstration. A full list of stations can be found [here](../station_listing) and downloaded as a csv file [here](https://github.com/AssessingSolar/radiationstations/blob/main/solarstations.csv).

## Overview of stations

To find the nearest station to a point of interest, check out the interactive map below. It's possible to click on a given station to get the station name.

In [18]:
import pandas as pd
import folium
from folium import plugins

df = pd.read_csv('solarstations.csv', sep=';', encoding='latin1')

EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"

# Create Folium map
m = folium.Map(
    location=[0, 15],
    zoom_start=1, min_zoom=1, max_bounds=True,
    #width=1000, height=525,
    control_scale=True,  # Adds distance scale in lower left corner
    tiles='openstreetmap',
)

# Add each station to the map
# Consider using apply instead of for loop to add stations in case of many stations
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station full name'] + ' - ' + row['Location'],
        tooltip=row['Abbreviation'],
        icon=folium.Icon(color='blue', icon='bolt', prefix='fa')
    ).add_to(m)

folium.raster_layers.TileLayer(EsriImagery, name='World imagery', attr=EsriAttribution).add_to(m)
folium.LayerControl(position='topright').add_to(m)

# Additional options and plugins
plugins.MiniMap(toggle_display=True, position='bottomleft').add_to(m)  # Add minimap to the map
plugins.Fullscreen(position='bottomright').add_to(m)  # Add full screen button to map
folium.LatLngPopup().add_to(m)  # Show latitude/longitude when clicking on the map
# plugins.MeasureControl(position='topleft').add_to(m)  # Add distance length measurement tool

# Show the map
m 

## Potential map features
Some of the future improvements could include
* coloring of the stations according to the network (e.g., BSRN, NREL, SRML, Enermena).
* toggling the visibility stations based on network
* shown an image of the station (url) when clicking on the station

* Also, the table of stations above will be made sortable and with clickable url's.

### Features not idea
It is also possible to have an field where a user can type in an address, however, I do not seem much use in this.

With many points on the map it becomes hard to see the individual stations. This could be adressed by using a [clustered point map](https://autogis-site.readthedocs.io/en/latest/notebooks/L5/02_interactive-map-folium.html#clustered-point-map).

Currently, the points on the map are 

## Acknowledgements
The map was created using [Folium](http://python-visualization.github.io/folium/). Check out this [tutorial](https://www.youtube.com/watch?v=t9Ed5QyO7qY&ab_channel=RyanNoonan) for making a interactive map.